# Populations
[binder](https://mybinder.org/v2/gh/hsolbrig/clinicalprofileexamples/master?filepath=examples%2Fpopulationandcohort.ipynb)

## "abstract" population definition
Asthma population is an abstract definition 

In [23]:

import requests
from jsonasobj import loads, as_json, JsonObj
from typing import Optional, Callable, List, Tuple

def read_group(id_: str) -> Optional[JsonObj]:
    """ Read the group referenced by id """
    resp = requests.get(f"http://hapi.clinicalprofiles.org/baseR4/Group/{id_}?_pretty=true&_format=_json")
    if resp.status_code == 200:
        return loads(resp.text)
    else:
        print(resp.reason)
        return None


In [24]:
asthma_pop = read_group("asthma-population")
print(as_json(asthma_pop))

{
   "resourceType": "Group",
   "id": "asthma-population",
   "meta": {
      "versionId": "2",
      "lastUpdated": "2019-06-21T00:07:01.542+00:00"
   },
   "text": {
      "status": "additional",
      "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">\n    <p>Patients having a diagnosis of asthma recorded on the problem list or as an encounter diagnosis.</p>\n    <p>This is an abstract profile that can be applied to different base populations</p>\n</div>"
   },
   "identifier": [
      {
         "system": "http://ncats.org/clinicalprofile/",
         "value": "asthma-population"
      }
   ],
   "type": "person",
   "actual": false,
   "name": "Patients diagnosed with asthma",
   "characteristic": [
      {
         "code": {
            "text": "encounter diagnosis"
         },
         "valueCodeableConcept": {
            "coding": [
               {
                  "system": "http://snomed.info/sct",
                  "code": "195967001",
                  "display": "Asth

A population has `actual` set to false and does not define a quantity

In [25]:
print(f"Actual = {asthma_pop.actual}")

Actual = False


The `text` and `name` are intended to provide a human readable description 
(we need to make sure we can include attribution for this)

In [26]:
from IPython.core.display import display, HTML
display(HTML(asthma_pop.text.div))
print('- ' * 20)
print(asthma_pop.name)

- - - - - - - - - - - - - - - - - - - - 
Patients diagnosed with asthma


Populations can also be defined in a (semi-) formal fashion.  Note that we still need to locate a code for "encounter diagnosis"...

In [27]:
print(as_json(asthma_pop.characteristic[0]))

{
   "code": {
      "text": "encounter diagnosis"
   },
   "valueCodeableConcept": {
      "coding": [
         {
            "system": "http://snomed.info/sct",
            "code": "195967001",
            "display": "Asthma"
         },
         {
            "system": "http://human-phenotype-ontology.org/",
            "code": "002099",
            "display": "Asthma"
         }
      ],
      "text": "asthma"
   }
}


## "actual" populations
There can be multiple actual populations that "realize" an abstract definition:

In [28]:
jhu_asthma = read_group("asthma")
print(as_json(jhu_asthma))

{
   "resourceType": "Group",
   "id": "asthma",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2019-06-21T00:06:43.604+00:00"
   },
   "text": {
      "status": "additional",
      "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\">\n    <p>All patients seen as either outpatients or inpatients at Johns Hopkins Hospital, JHCP, and Bayview between \n3/1/2016 and 3/1/2018 having a diagnosis of asthma recorded on the problem list of as an encounter diagnosis</p>\n</div>"
   },
   "identifier": [
      {
         "system": "http://ncats.org/clinicalprofile/",
         "value": "asthma"
      }
   ],
   "type": "person",
   "actual": true,
   "name": "JHU Patients diagnosed with asthma 3/1/2016 to 3/1/2018",
   "quantity": 61449,
   "characteristic": [
      {
         "code": {
            "text": "encounter diagnosis"
         },
         "valueCodeableConcept": {
            "coding": [
               {
                  "system": "http://hl7.org/fhir/sid/icd-9-cm",
        

In the above case, `actual` is True and we include the set of codes that were used to retrieve the data.

## Cohort definitions
Cohorts are partitioned into "abstract" and "actual" as well.

In [29]:
def filtr_group(filtr: Callable[[JsonObj], bool]) -> List[Tuple[str, str]]:
    """ Query groups using filtr """
    url = "http://hapi.clinicalprofiles.org/baseR4/Group?_format=json&_pretty=true"
    rval = []
    while url:
        resp = requests.get(url)
        url = None
        if resp.status_code == 200:
            bundle = loads(resp.text)
            for r in bundle.link:
                if r.relation == 'next':
                    url = r.url
                    break
            for e in bundle.entry:
                if filtr(e.resource):
                    rval.append((e.resource.id, e.resource.name))
    return rval

In [34]:
female_asthma_45_64 = read_group('asthma-Female-ANY-45-64-population')
print(as_json(female_asthma_45_64))

{
   "resourceType": "Group",
   "id": "asthma-Female-ANY-45-64-population",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2019-06-21T00:07:17.846+00:00"
   },
   "active": true,
   "type": "person",
   "actual": false,
   "name": "asthma-Female-ANY-45-64-population",
   "characteristic": [
      {
         "code": {
            "text": "base population"
         },
         "valueReference": {
            "reference": "Group/asthma-population",
            "type": "Group",
            "display": "patients with asthma"
         }
      },
      {
         "code": {
            "text": "patient age"
         },
         "valueRange": {
            "low": {
               "value": 45.0,
               "unit": "years"
            },
            "high": {
               "value": 64.0,
               "unit": "years"
            }
         }
      },
      {
         "code": {
            "text": "patient gender"
         },
         "valueCodeableConcept": {
            "coding

In [35]:
female_asthma_45_64 = read_group('asthma-Female-ANY-45-64.')
print(as_json(female_asthma_45_64))

{
   "resourceType": "Group",
   "id": "asthma-Female-ANY-45-64.",
   "meta": {
      "versionId": "1",
      "lastUpdated": "2019-06-21T00:07:18.902+00:00"
   },
   "active": true,
   "type": "person",
   "actual": true,
   "name": "asthma-Female-ANY-45-64.",
   "managingEntity": {
      "reference": "Organization/JHU",
      "type": "Organization"
   },
   "characteristic": [
      {
         "code": {
            "text": "base population"
         },
         "valueReference": {
            "reference": "Group/asthma",
            "type": "Group",
            "display": "patients with asthma"
         }
      },
      {
         "code": {
            "text": "patient age"
         },
         "valueRange": {
            "low": {
               "value": 45.0,
               "unit": "years"
            },
            "high": {
               "value": 64.0,
               "unit": "years"
            }
         }
      },
      {
         "code": {
            "text": "patient gender"
 

In [57]:
def realization_of(abstract_id: str) -> List[Tuple[str, str]]:
    def test_base(group: JsonObj) -> bool:
        if getattr(group, 'characteristic', None):
            for c in group.characteristic:
                if c.code.text == "base population" and c.valueReference.reference == f"Group/{abstract_id}":
                    return True
        return False
    return test_base

realizations = filtr_group(realization_of('asthma'))
display(HTML("<h1>Realization of asthma populations</h1>"))
print('\n'.join(e[0] for e in sorted(realizations, key=lambda e: e[0])))
   

asthma-ANY-ANY-18-44.
asthma-ANY-ANY-45-64.
asthma-ANY-ANY-65-74.
asthma-ANY-ANY-75-200.
asthma-ANY-ANY-ANY
asthma-ANY-Black-or-African-American-18-44.
asthma-ANY-Black-or-African-American-45-64.
asthma-ANY-Black-or-African-American-65-74.
asthma-ANY-Black-or-African-American-75-200.
asthma-ANY-Black-or-African-American-ANY
asthma-ANY-Other-18-44.
asthma-ANY-Other-45-64.
asthma-ANY-Other-65-74.
asthma-ANY-Other-75-200.
asthma-ANY-Other-ANY
asthma-ANY-White-or-Caucasian-18-44.
asthma-ANY-White-or-Caucasian-45-64.
asthma-ANY-White-or-Caucasian-65-74.
asthma-ANY-White-or-Caucasian-75-200.
asthma-ANY-White-or-Caucasian-ANY
asthma-Female-ANY-18-44.
asthma-Female-ANY-45-64.
asthma-Female-ANY-65-74.
asthma-Female-ANY-75-200.
asthma-Female-ANY-ANY
asthma-Female-Black-or-African-American-18-44.
asthma-Female-Black-or-African-American-45-64.
asthma-Female-Black-or-African-American-65-74.
asthma-Female-Black-or-African-American-75-200.
asthma-Female-Black-or-African-American-ANY
asthma-Female-Oth